In [1]:
import requests 
from lxml import etree 
import re
import time
import pandas as pd
from pandas import DataFrame
import os
from openpyxl import load_workbook

In [2]:
tp=pd.read_excel(r"C:\Users\Administrator\Desktop\dailywork\1.xlsx",sheet_name=None)
null=pd.DataFrame()

In [3]:
data={}
'''
get all sheet names:
df = pd.read_excel('自己的Excel文件路径.xlsx', sheet_name=None)  # 路径注意转义,会生成字典文件，字典中每个值都是dataframe格式的 
for i in df.keys():
   print(i)
or
df = pd.read_excel('自己的Excel文件路径.xlsx', sheet_name=None)
print(list(df))nihao
'''

"\nget all sheet names:\ndf = pd.read_excel('自己的Excel文件路径.xlsx', sheet_name=None)  # 路径注意转义,会生成字典文件，字典中每个值都是dataframe格式的 \nfor i in df.keys():\n   print(i)\nor\ndf = pd.read_excel('自己的Excel文件路径.xlsx', sheet_name=None)\nprint(list(df))nihao\n"

In [8]:
for i in tp.keys():
    NO=1
    data[i+'-'+str(NO)]=pd.DataFrame(columns=['序号', '货物或应税劳务、服务名称','计量单位','规格型号','数量','金额','税率','商品税目','折扣金额','税额','折扣税额','折扣率','单价','价格方式','税收分类编码版本号','税收分类编码','企业商品编码','使用优惠政策标识','零税率标识','优惠政策说明','中外合作油气田标识'])
    sheet_name=i
    a=tp[i]
    length=a.shape[0]-1
    section=0
    total=0
    mark=0
    for j in range(length):
        tag=a.iloc[j,1]
        model=a.iloc[j,2]
        number=a.iloc[j,4]
        price_sum=round(float(a.iloc[j,9]),2)
        #tax = '%.2f' % (round(price_sum/1.13*0.13,2))
        tax=round(price_sum/1.13*0.13,2)
        price_per=round(price_sum/number,2)
        section=round(section+price_sum,2)
        if section < 11300000:
            data[i+'-'+str(NO)].at[j-mark,'序号']=j-mark+1
            data[i+'-'+str(NO)].at[j-mark,'货物或应税劳务、服务名称']=tag
            data[i+'-'+str(NO)].at[j-mark,'计量单位']='套'
            data[i+'-'+str(NO)].at[j-mark,'规格型号']=model
            data[i+'-'+str(NO)].at[j-mark,'数量']=number
            data[i+'-'+str(NO)].at[j-mark,'金额']='%.2f' % price_sum
            data[i+'-'+str(NO)].at[j-mark,'税率']='0.13'
            data[i+'-'+str(NO)].at[j-mark,'税额']='%.2f' % tax
            data[i+'-'+str(NO)].at[j-mark,'单价']='%.2f' % price_per
            data[i+'-'+str(NO)].at[j-mark,'价格方式']='1'
            data[i+'-'+str(NO)].at[j-mark,'税收分类编码版本号']='20'
            if tag !='洗衣机组件':
                data[i+'-'+str(NO)].at[j-mark,'税收分类编码']='1090421020000000000'
            else:
                data[i+'-'+str(NO)].at[j-mark,'税收分类编码']='1090421030000000000'
            data[i+'-'+str(NO)].at[j-mark,'使用优惠政策标识']='0'
            data[i+'-'+str(NO)].at[j-mark,'中外合作油气田标识']='0'
        else:
            total+=section-price_sum
            section=price_sum
            mark=j
            NO+=1
            data[i+'-'+str(NO)]=pd.DataFrame(columns=['序号', '货物或应税劳务、服务名称','计量单位','规格型号','数量','金额','税率','商品税目','折扣金额','税额','折扣税额','折扣率','单价','价格方式','税收分类编码版本号','税收分类编码','企业商品编码','使用优惠政策标识','零税率标识','优惠政策说明','中外合作油气田标识'])
            data[i+'-'+str(NO)].at[0,'序号']=1
            data[i+'-'+str(NO)].at[0,'货物或应税劳务、服务名称']=tag
            data[i+'-'+str(NO)].at[0,'计量单位']='套'
            data[i+'-'+str(NO)].at[0,'规格型号']=model
            data[i+'-'+str(NO)].at[0,'数量']=number
            data[i+'-'+str(NO)].at[0,'金额']='%.2f' % price_sum
            data[i+'-'+str(NO)].at[0,'税率']='0.13'
            data[i+'-'+str(NO)].at[0,'税额']='%.2f' % tax
            data[i+'-'+str(NO)].at[0,'单价']='%.2f' % price_per
            data[i+'-'+str(NO)].at[0,'价格方式']='1'
            data[i+'-'+str(NO)].at[0,'税收分类编码版本号']='20'
            if tag =='冰箱组件'or '冷柜组件':
                data[i+'-'+str(NO)].at[0,'税收分类编码']='1090421020000000000'
            else:
                data[i+'-'+str(NO)].at[0,'税收分类编码']='1090421030000000000'
            data[i+'-'+str(NO)].at[0,'使用优惠政策标识']='0'
            data[i+'-'+str(NO)].at[0,'中外合作油气田标识']='0'

    name=i+".xlsx"
    null.to_excel(name)
    writer=pd.ExcelWriter(name)
    for r in range(1,NO):
        data[i+'-'+str(r)].to_excel(writer,sheet_name=i+'-'+str(r))
    writer.save()
    writer.close()

In [65]:
print(data)

{'南通1-1':     序号 货物或应税劳务、服务名称 计量单位             规格型号    数量           金额    税率 商品税目 折扣金额  \
0    0         冰箱组件    套        5GW45DB-T    28      38515.1  0.13  NaN  NaN   
1    1         冰箱组件    套  BCD-300EGX4SU-T   976  1.08755e+06  0.13  NaN  NaN   
2    2         冰箱组件    套  BCD-308WEGX4S-T     1      1302.03  0.13  NaN  NaN   
3    3         冰箱组件    套  BCD-309WD4EGL-T     5      6572.15  0.13  NaN  NaN   
4    4         冰箱组件    套  BCD-311WD4EBL-T  1024   1.4878e+06  0.13  NaN  NaN   
..  ..          ...  ...              ...   ...          ...   ...  ...  ...   
69  69        洗衣机组件    套    XQB70-10D0B-T  1066       507405  0.13  NaN  NaN   
70  70        洗衣机组件    套    XQB90-1215G-T     2      1446.24  0.13  NaN  NaN   
71  71        洗衣机组件    套    BCD-207GB3S-T     2      1422.92  0.13  NaN  NaN   
72  72        洗衣机组件    套   BCD-400EGX5S-T     4      4959.72  0.13  NaN  NaN   
73  73        洗衣机组件    套  BCD-602WEGY5S-T     6      12238.4  0.13  NaN  NaN   

         税额  ...  折扣率       单

In [32]:
length=a.shape[0]-1
print(length)

    print(a.iloc[3,9])
    print(a.iloc[74,4])

74


In [6]:
a=tp['score']

In [36]:
price=float(a.iloc[3,9])
type(price)

float

In [50]:
df = pd.DataFrame(columns=['a', 'b'])
df.at[0, 'a'] = 'your_data'
print(df)


           a    b
0  your_data  NaN


In [8]:
pip install Excelwriter

Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement Excelwriter (from versions: none)
ERROR: No matching distribution found for Excelwriter
